In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator='\n',
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")
docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()

# cache_dir에 있는 embed를 확인하고 업으면 OpenAIEmbeddings 사용
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

# retriever : class의 interface. document를 retrieve
# from vector store, cloud 등
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",
    retriever=vectorstore.as_retriever(),
)

chain.run("Describe Victory Mansions.")

"Victory Mansions, as described in the additional context, is a part of the grim and deteriorating landscape of London, the chief city of Airstrip One in the province of Oceania. The apartment complex stands in stark contrast to the oppressive government buildings, such as the Ministry of Truth, which symbolize the ruling Party's power. The exterior of Victory Mansions reflects the desolation and decay of the surrounding area, with little color and torn posters plastered everywhere. The prominent posters, including one with Big Brother's face, serve as constant reminders of the government's surveillance and control. In addition to the deteriorating appearance, the inhabitants of Victory Mansions must also live under constant surveillance, as any sound or movement could be monitored by the Thought Police. The building's crumbling walls, broken windows, and general state of disrepair further contribute to the bleak and austere environment, emphasizing the oppressive nature of the totalit